In [1]:
!pip install -q fastapi "uvicorn[standard]==0.29.0" nest_asyncio pyngrok streamlit crewai crewai_tools langchain-groq langfuse fpdf2 groq --upgrade tokenizers
!pip install uvicorn==0.23.2
import nest_asyncio
nest_asyncio.apply()

  Using cached uvicorn-0.23.2-py3-none-any.whl.metadata (6.2 kB)
Using cached uvicorn-0.23.2-py3-none-any.whl (59 kB)
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.29.0
    Uninstalling uvicorn-0.29.0:
      Successfully uninstalled uvicorn-0.29.0


In [2]:
# STEP 2: Set environment variables
import os

os.environ["GEOAPIFY_API_KEY"] = "87aea5b7db954ec49acf7087747faf7b"
os.environ["WEATHER_API_KEY"] = "d98414f9ced5434a962203029251907"
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-9a6c3825-aceb-4021-aabd-ddc62081b4af"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-ba46752d-6afc-41e9-a577-4f5a386aa81d"
os.environ["GROQ_API_KEY"] = "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"
os.environ["LANGFUSE_HOST"]= "https://cloud.langfuse.com"

In [27]:
import requests
import os

def get_places(city, interest, limit=3):
    """Fetches places and their weather using the Geoapify Geocode API."""
    api_key = os.getenv('GEOAPIFY_API_KEY')
    if not api_key:
        return {"error": "Missing GEOAPIFY_API_KEY"}

    search_query = f"{interest} in {city}"
    url = f"https://api.geoapify.com/v1/geocode/search?text={search_query}&limit={limit}&apiKey={api_key}"

    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if not data.get('features'):
            return {"error": f"No places found for '{interest}' in {city}"}

        for feature in data['features']:
            properties = feature.get('properties', {})
            lat = properties.get('lat')
            lon = properties.get('lon')
            if lat and lon:
                print(f"Fetching weather for {properties.get('formatted', 'Unknown')} from wttr.in")
                weather_data = get_weather(f"{lat},{lon}")
                properties['weather'] = weather_data
        return data

    except Exception as e:
        return {"error": f"Failed to get places for '{interest}' in '{city}': {e}"}


# --- Test the updated function ---
result = get_places("Paris", "museums")
if result.get('features'):
    print(f"Found {len(result['features'])} places:")
    for place in result['features']:
        name = place.get('properties', {}).get('formatted', 'Unknown')
        weather = place.get('properties', {}).get('weather', 'No weather data')
        print(f"  • {name}\n    Weather: {weather}")
else:
    print(f"Error or no places found: {result.get('error', 'Unknown issue')}")


def get_weather(city):
    """Fetches weather from wttr.in"""
    url = f"https://wttr.in/{city}?format=j1"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        return {"error": str(e)}

Fetching weather for Paris, IDF, France from wttr.in
Fetching weather for Paris (Paris), TX, United States of America from wttr.in
Fetching weather for Paris, TN, United States of America from wttr.in
Found 3 places:
  • Paris, IDF, France
    Weather: {'current_condition': [{'FeelsLikeC': '18', 'FeelsLikeF': '65', 'cloudcover': '75', 'humidity': '94', 'localObsDateTime': '2025-07-27 09:25 AM', 'observation_time': '07:25 AM', 'precipInches': '0.0', 'precipMM': '0.0', 'pressure': '1016', 'pressureInches': '30', 'temp_C': '18', 'temp_F': '65', 'uvIndex': '1', 'visibility': '10', 'visibilityMiles': '6', 'weatherCode': '296', 'weatherDesc': [{'value': 'Light drizzle and rain'}], 'weatherIconUrl': [{'value': ''}], 'winddir16Point': 'NW', 'winddirDegree': '306', 'windspeedKmph': '12', 'windspeedMiles': '7'}], 'nearest_area': [{'areaName': [{'value': 'Paris'}], 'country': [{'value': 'France'}], 'latitude': '48.867', 'longitude': '2.333', 'population': '2110694', 'region': [{'value': 'Ile-de-F

In [30]:
# pdf_export.py
from fpdf import FPDF
import json

def export_to_pdf(itinerary_text, filename="itinerary.pdf"):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Get page width minus margins
        page_width = pdf.w - 2 * pdf.l_margin

        for line in itinerary_text.split("\n"):
            if line.strip():  # Skip empty lines
                if line.startswith("Weather:"):
                    try:
                        weather_json = json.loads(line.replace("Weather: ", ""))
                        current = weather_json.get('current_condition', [{}])[0]
                        weather_desc = current.get('weatherDesc', [{}])[0].get('value', 'N/A')
                        temp_c = current.get('temp_C', 'N/A')
                        feels_like_c = current.get('FeelsLikeC', 'N/A')

                        weather_str = f"Weather: {weather_desc}, Temp: {temp_c}°C, Feels like: {feels_like_c}°C"
                        pdf.ln(5) # Add a line break
                        pdf.multi_cell(page_width, 10, weather_str)
                    except (json.JSONDecodeError, IndexError):
                        pdf.multi_cell(page_width, 10, line) # fallback to raw
                else:
                    pdf.multi_cell(page_width, 10, line)

        pdf.output(filename)
        return {"success": True, "filename": filename}

    except Exception as e:
        return {"success": False, "error": str(e)}

# Test the export_to_pdf function with the top 1 place and its weather
result = get_places("Paris", "museums", limit=1)
if result.get('features'):
    place = result['features'][0]
    name = place.get('properties', {}).get('formatted', 'Unknown')
    weather = place.get('properties', {}).get('weather', 'No weather data')

    # Format the text for the PDF
    itinerary_text = f"Top Attraction: {name}\nWeather: {json.dumps(weather)}"

    # Export to PDF
    pdf_result = export_to_pdf(itinerary_text, filename="top_place_itinerary.pdf")
    if pdf_result['success']:
        print(f"✅ Successfully exported to {pdf_result['filename']}")
    else:
        print(f"❌ Failed to export to PDF: {pdf_result['error']}")
else:
    print(f"Error or no places found: {result.get('error', 'Unknown issue')}")

Fetching weather for Paris, IDF, France from wttr.in
✅ Successfully exported to top_place_itinerary.pdf


/tmp/ipython-input-30-3665076329.py:9: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)


In [34]:
import os
from crewai import Agent
from crewai.tools import BaseTool
from textwrap import dedent
from langchain_groq import ChatGroq
from typing import Any
import json

class GetPlacesTool(BaseTool):
    name: str = "get_places"
    description: str = "A tool to get places and their weather based on a city and interest."

    def _run(self, city: Any, interest: Any) -> str:
        if isinstance(city, dict) and "city" in city:
            city = city["city"]
        if isinstance(interest, dict) and "interest" in interest:
            interest = interest["interest"]

        places_data = get_places(city, interest)

        if "error" in places_data:
            return f"Error finding places: {places_data['error']}"

        if not places_data.get("features"):
            return f"No places found for {interest} in {city}."

        # Format the output into a clean string for the LLM
        formatted_places = []
        for place in places_data["features"]:
            properties = place.get("properties", {})
            name = properties.get("formatted", "Unknown place")
            weather = properties.get("weather", {})
            current = weather.get("current_condition", [{}])[0]
            weather_desc = current.get("weatherDesc", [{}])[0].get("value", "N/A")
            temp_c = current.get("temp_C", "N/A")
            feels_like_c = current.get("FeelsLikeC", "N/A")

            weather_summary = f"Weather: {weather_desc}, Temp: {temp_c}°C, Feels like: {feels_like_c}°C"
            formatted_places.append(f"{name} ({weather_summary})")

        return ", ".join(formatted_places)

get_places_tool = GetPlacesTool()


# Configure Groq LLM properly
groq_llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="groq/llama3-8b-8192",
    temperature=0.7
)

# --- Define Your Agents ---

planner_agent = Agent(
    role="Planner",
    goal="List top attractions in the city with their weather, based on user's interests",
    backstory="An expert in recommending attractions and their current weather.",
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
    tools=[get_places_tool]
)

class WeatherTool(BaseTool):
    name: str = "get_weather"
    description: str = "A tool to get the weather forecast for a city."

    def _run(self, city: Any) -> str:
        if isinstance(city, dict) and "city" in city:
            city = city["city"]
        try:
            weather_data = get_weather(city)
            if "error" in weather_data:
                return f"Error: {weather_data['error']}"

            # Extract relevant info to reduce tokens
            current = weather_data.get("current_condition", [{}])[0]
            weather_desc = current.get("weatherDesc", [{}])[0].get("value", "N/A")
            temp_c = current.get("temp_C", "N/A")
            feels_like_c = current.get("FeelsLikeC", "N/A")

            return f"Current weather in {city}: {weather_desc}, Temp: {temp_c}°C, Feels like: {feels_like_c}°C"

        except Exception as e:
            return f"Error: {e}"

weather_tool = WeatherTool()

weather_agent = Agent(
    role="Weather Forecaster",
    goal="Provide a 1-day weather forecast for the selected city",
    backstory="A meteorologist providing short-term forecasts.",
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
    tools=[weather_tool]
)

coordinator_agent = Agent(
    role="Coordinator",
    goal="Compile a complete travel itinerary combining attractions and weather",
    backstory="An experienced travel itinerary designer.",
    verbose=True,
    allow_delegation=True,
    llm=groq_llm
)

print("✅ All agents created successfully")

✅ All agents created successfully


In [35]:
# Test if agents are properly configured
def test_agents():
    agents = [
        ("Planner", planner_agent),
        ("Weather", weather_agent),
        ("Coordinator", coordinator_agent)
    ]

    for name, agent in agents:
        try:
            print(f"✅ {name} agent: {agent.role}")
        except Exception as e:
            print(f"❌ {name} agent failed: {e}")

test_agents()

✅ Planner agent: Planner
✅ Weather agent: Weather Forecaster
✅ Coordinator agent: Coordinator


In [60]:
from crewai import Task, Crew, Process
from textwrap import dedent
import time

# Fix 1: Uncomment and import agents
#from agents import planner_agent, weather_agent, coordinator_agent

# Fix 2: Add function with input parameters
def generate_itinerary(city, interests, max_distance_km=10):
    """Generate itinerary with proper inputs"""

    # Step 1: Create tasks with city and interests
    planner_task = Task(
        description=dedent(f"""\
            Identify and list the top attractions in {city} with their weather, based on these interests: {interests}.
            The attractions should be within a {max_distance_km} km radius.
            Only include tourist-friendly places (e.g., museums, parks, historic sites).
        """),
        expected_output="A markdown list of top 3 attractions with name, short description, distance, and weather.",
        agent=planner_agent
    )

    # Step 2: Create Crew
    crew = Crew(
        agents=[planner_agent],
        tasks=[planner_task],
        process=Process.sequential,
        verbose=True
    )

    # Step 3: Run Crew
    print(f"🚀 Generating itinerary for {city}...")
    crew.kickoff()

    successful_tasks = [task for task in crew.tasks if task.output]

    if successful_tasks:
        print("\n\n---Task Outputs---")
        for task in successful_tasks:
            print(f"Task: {task.description}")
            print(f"Output: {task.output}")
        return {"success": True, "tasks": successful_tasks}
    else:
        print("All agents failed to produce an output.")
        return {"success": False, "error": "All agents failed to produce an output."}


# Test it
print("Testing itinerary generation:")
result = generate_itinerary("Paris", "art, museums")

Testing itinerary generation:
🚀 Generating itinerary for Paris...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6e02e04c-5718-4b04-9ae7-610898525e1a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Task: Identify and list the top attractions in Paris with their weather, based on these interests: art,        │
│  museums.                                                                                                       │
│  The attractions should be within a 10 km radius.                                                               │
│  Only include tourist-friendly places (e.g., museums, parks, historic sites).                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Fetching weather for Place description, Aberdeen, WA 98520, United States of America from wttr.in

Fetching weather for MAN Art, Rue de la Tombe Issoire, 75014 Paris, France from wttr.in

Fetching weather for Art + art galerie, 28 Rue de la Fontaine au Roi, 75011 Paris, France from wttr.in

KeyboardInterrupt: 

In [4]:
!pip install "fastapi[standard]"

  Using cached uvloop-0.21.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 9.0 MB/s eta 0:00:00
Using cached uvloop-0.21.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.6/950.6 kB 36.6 MB/s eta 0:00:00


In [21]:
import time
import subprocess
import requests
from pyngrok import ngrok

def start_fastapi():
    # Run uvicorn as a subprocess, capturing output for debugging
    process = subprocess.Popen(
        ["uvicorn", "__main__:app", "--host", "0.0.0.0", "--port", "8000"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    return process

# Start server
fastapi_process = start_fastapi()

# Wait and verify server is up
max_attempts = 12
server_up = False
for attempt in range(max_attempts):
    try:
        response = requests.get("http://localhost:8000/", timeout=2)
        if response.status_code == 200:
            server_up = True
            print("✅ Server is up and running.")
            break
    except requests.exceptions.RequestException:
        print(f"⏳ Attempt {attempt + 1}: Server not yet available, retrying...")
        time.sleep(2)

if not server_up:
    print("❌ Failed to verify server is running. Check uvicorn logs.")
else:
    # Set ngrok auth token (replace with your actual token string)
    ngrok.set_auth_token("3065jfdyeiW8ZJ4cHdXILkfypxT_5aZygde3TRFzprouNHQ2T")

    # Connect ngrok with retries
    for attempt in range(3):
        try:
            tunnel = ngrok.connect(8000, "http")
            public_url = tunnel.public_url
            print(f"🚀 Your FastAPI app is live at: {public_url}")
            break
        except Exception as e:
            print(f"⚠️ Ngrok connection attempt {attempt + 1} failed: {e}")
            time.sleep(5)
    else:
        print("❌ Failed to establish ngrok tunnel after multiple retries.")



⏳ Attempt 1: Server not yet available, retrying...
⏳ Attempt 2: Server not yet available, retrying...
⏳ Attempt 3: Server not yet available, retrying...
⏳ Attempt 4: Server not yet available, retrying...
⏳ Attempt 5: Server not yet available, retrying...
⏳ Attempt 6: Server not yet available, retrying...
⏳ Attempt 7: Server not yet available, retrying...
⏳ Attempt 8: Server not yet available, retrying...
⏳ Attempt 9: Server not yet available, retrying...
⏳ Attempt 10: Server not yet available, retrying...
⏳ Attempt 11: Server not yet available, retrying...
⏳ Attempt 12: Server not yet available, retrying...
❌ Failed to verify server is running. Check uvicorn logs.


In [22]:
%%writefile app.py
import streamlit as st
import requests

FASTAPI_URL = public_url  # Default fallback

st.title("🗺️ AI Travel Planner")
st.write("Plan your trip by selecting a city, interest, and distance.")

city = st.text_input("City", "Paris")
interest = st.text_input("Interest", "museums")
distance = st.slider("Max Distance (km)", 1, 20, 5)

if st.button("Plan Trip"):
    params = {
        "city": city,
        "interest": interest,
        "max_distance_km": distance
    }

    try:
        st.write(f"🔍 Sending request to {FASTAPI_URL}/api/plan-trip with: {params}")
        response = requests.get(f"{FASTAPI_URL}/api/plan-trip", params=params, timeout=5)
        response.raise_for_status()

        itinerary = response.json().get("itinerary", "No itinerary found.")
        st.text_area("Generated Itinerary", str(itinerary), height=300)

        pdf_url = f"{FASTAPI_URL}/download-pdf"
        st.markdown(f"<a href='{pdf_url}' target='_blank' download>📄 Download Itinerary PDF</a>", unsafe_allow_html=True)

    except Exception as e:
        st.warning("⚠️ Backend not reachable. Showing mock itinerary instead.")
        mock_itinerary = f"""Explore top {interest} in {city} within {distance} km:

- Mock Museum A
- Mock Museum B
- Mock Museum C
"""
        st.text_area("Generated Itinerary (Mock)", mock_itinerary, height=300)


Overwriting app.py


In [23]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.27.234.198:8501

  Stopping...
  Stopping...
